In [1]:
from tqdm import tqdm
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from rouge import Rouge
import torch
import csv
import random
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertModel, BertTokenizer

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x1072ce1b0>>
Traceback (most recent call last):
  File "/Users/matteorigat/PycharmProjects/nlp-project/.venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [2]:
saved_model = "Models/colab_model_3"
model_name = "gpt2"

In [3]:
# Load the GPT tokenizer.
tokenizer = GPT2Tokenizer.from_pretrained(model_name, bos_token='[BOS]', eos_token='[EOS]', pad_token='[PAD]')
# add special tokens for title, ingredients and instruction seperator
special_tokens_dict = {'additional_special_tokens': ['[STEPS]']} #'[INGREDIENTS]', 
# check the number of special tokens
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
print('We have added', num_added_toks, 'tokens')

We have added 1 tokens


In [4]:
# Load the trained GPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained(saved_model)

# Ensure the model is on the right device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50261, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50261, bias=False)
)

In [5]:
def generate_recipe(ingredients, model, tokenizer, max_length=300, temperature=0.1, top_k=50, top_p=0.1):
    input_text = '[BOS]' + ingredients + '[STEPS]'
    input_ids = tokenizer(input_text, return_tensors='pt').input_ids.to(device)
    
    output = model.generate(
        input_ids,
        max_length=max_length,
        temperature=temperature, # Lower values make the model more confident (less random), while higher values increase randomness.
        top_k=top_k,  #Increase to consider more tokens, decrease to restrict the model’s choices.
        top_p=top_p,  # Increase to allow more diversity, decrease to make the model more conservative.
        num_beams=5,
        no_repeat_ngram_size=2,
        num_return_sequences=1,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True
    )
    
    recipe = tokenizer.decode(output[0], skip_special_tokens=False)
    
    # Replace lowercase special tokens with uppercase
    recipe = recipe.replace('[bos]', '[BOS]').replace('[steps]', '[STEPS]').replace('[eos]', '[EOS]')
    
    recipe = recipe.split('[EOS]', 1)[0] + '[EOS]'
        
    return recipe

In [6]:
def print_highlighted(generated_recipe, ingredients):
    recipe=generated_recipe
    ingredients_list = [ing.strip().lower() for ing in ingredients.split(',')]
    for ingredient in ingredients_list:
        recipe = recipe.replace(ingredient, f'\033[91m{ingredient}\033[0m')
    return recipe

In [7]:
# Example usage
ingredients = "pasta, tomato, fish"

generated_recipe = generate_recipe(ingredients, model, tokenizer)
    
print(print_highlighted(generated_recipe, ingredients))
print("\n", len(generated_recipe) - len(ingredients))

[BOS]pasta, tomato, fish [STEPS], mozzarella cheese[STEPS]preheat oven to 400 degrees fahrenheit, spray a baking sheet with non-stick cooking spray, arrange fish in a single layer, sprinkle with salt and pepper, top with tomatoes, onion, and cheese, bake for 15 to 20 minutes or until fish flakes easily with a fork[EOS]

 301


# Evaluate the generated recipe

In [8]:
# load and also preprocess the raw data
def load_preprocess_raw_data(raw_data):
    recipe_instances = []

    with open(raw_data, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            # Extract relevant fields from CSV row
            #name = row['name'].lower().replace('"', '')  # Remove any extra quotes
            ingredients = row['ingredients'].lower().replace('\'', '').replace('[', '').replace(']', '')
            instructions = row['steps'].lower().replace('\'', '').replace('[', '').replace(']', '')

            # Prepare recipe instance string
            recipe_instance = '[BOS]' + ingredients + '[STEPS]' + instructions + '[EOS]'  #+name+'[INGREDIENTS]'

            # Limit length to 2000 characters as per your function
            if len(recipe_instance) <= 3000:
                recipe_instances.append(recipe_instance)

    return recipe_instances

In [9]:
# create text list for dataset
# https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions/data
recipe_list = load_preprocess_raw_data("dataset/RAW_recipes.csv")
recipe_list = random.sample(recipe_list, int(0.01 * len(recipe_list)))

In [10]:
# Initialize models and tokenizers
model_name_bert = 'bert-base-uncased'
tokenizer_bert = BertTokenizer.from_pretrained(model_name_bert)
model_bert = BertModel.from_pretrained(model_name_bert)

rouge = Rouge()


# Function to calculate ROUGE-L F1 score
def calculate_rouge_score(text1, text2):
    scores = rouge.get_scores(text1, text2)
    rouge_l_f1 = scores[0]['rouge-l']['f']
    return rouge_l_f1


# Function to get GPT-2 embeddings
def get_gpt2_embedding(text, model, tokenizer):
    input_ids = tokenizer.encode(text, return_tensors='pt')
    with torch.no_grad():
        outputs = model(input_ids)
    hidden_states = outputs[0]
    pooled_embedding = torch.mean(hidden_states, dim=1)
    return pooled_embedding


# Function to calculate cosine similarity for GPT-2 embeddings
def calculate_gpt2_similarity(text1, text2, model, tokenizer):
    embedding1 = get_gpt2_embedding(text1, model, tokenizer)
    embedding2 = get_gpt2_embedding(text2, model, tokenizer)
    similarity = cosine_similarity(embedding1, embedding2).item()
    return similarity


# Function to encode text for BERT
def encode_text(text, tokenizer):
    input_ids = tokenizer.encode(text, return_tensors='pt', max_length=512, truncation=True)
    return input_ids


# Function to calculate BERT embeddings
def get_bert_embedding(input_ids, model):
    with torch.no_grad():
        outputs = model(input_ids)
        last_hidden_state = outputs.last_hidden_state
        pooled_embedding = torch.mean(last_hidden_state, dim=1)
    return pooled_embedding


# Function to calculate cosine similarity for BERT embeddings
def calculate_bert_similarity(text1, text2, model, tokenizer):
    input1 = encode_text(text1, tokenizer)
    input2 = encode_text(text2, tokenizer)
    embedding1 = get_bert_embedding(input1, model)
    embedding2 = get_bert_embedding(input2, model)
    similarity = cosine_similarity(embedding1.cpu(), embedding2.cpu()).item()
    return similarity


# Function to evaluate generated recipe against a list of real recipes
def evaluate_generated_recipe(generated_recipe, real_recipes):
    rouge_scores = []
    gpt2_similarities = []
    bert_similarities = []

    for real_recipe in tqdm(real_recipes, desc="Evaluating recipes"):
        rouge_score = calculate_rouge_score(generated_recipe, real_recipe)
        gpt2_similarity = calculate_gpt2_similarity(generated_recipe, real_recipe, model, tokenizer)
        bert_similarity = calculate_bert_similarity(generated_recipe, real_recipe, model_bert, tokenizer_bert)

        rouge_scores.append(rouge_score)
        gpt2_similarities.append(gpt2_similarity)
        bert_similarities.append(bert_similarity)

    # Calculate average scores
    avg_scores = [(sum(scores) / len(scores)) for scores in zip(rouge_scores)]
    #, gpt2_similarities, bert_similarities

    # Find index of recipe with maximum average score
    max_index = avg_scores.index(max(avg_scores))

    return real_recipes[max_index], rouge_scores[max_index], gpt2_similarities[max_index], bert_similarities[max_index]

In [11]:
best_recipe = evaluate_generated_recipe(generated_recipe, recipe_list)

print("Generated Recipe:")
print(print_highlighted(generated_recipe, ingredients))
print("\nMost Similar Real Recipe:")
print(print_highlighted(best_recipe[0], ingredients), "\n\nrouge-l f1:", best_recipe[1], "\nGPT-2 similarity:",
      best_recipe[2], "\nBERT similarity:", best_recipe[3])

Evaluating recipes: 100%|██████████| 2312/2312 [14:11<00:00,  2.71it/s]

Generated Recipe:
[BOS]pasta, tomato, fish [STEPS], mozzarella cheese[STEPS]preheat oven to 400 degrees fahrenheit, spray a baking sheet with non-stick cooking spray, arrange fish in a single layer, sprinkle with salt and pepper, top with tomatoes, onion, and cheese, bake for 15 to 20 minutes or until fish flakes easily with a fork[EOS]

Most Similar Real Recipe:
[BOS]red snapper, butter, parsley, fresh ground pepper, oranges[STEPS]preheat oven to 400 f, place the fish in a large baking pan, dot each piece of fish well with butter and sprinkle with parsley and pepper, lay the orange slices over the fish, bake for 20 minutes, reduce heat to 350 f, , and bake for 20 minutes more or until the fish flakes at the touch of a fork[EOS] 

rouge-l f1: 0.39130434288516075 
GPT-2 similarity: 0.9997220635414124 
BERT similarity: 0.9518119096755981


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer


# Function to extract ingredients from a recipe
def extract_ingredients(recipe):
    start = recipe.find('[BOS]') + len('[BOS]')
    end = recipe.find('[STEPS]')
    ingredients = recipe[start:end].strip()
    return ingredients


# Function to calculate cosine similarity for ingredient lists
def calculate_ingredient_similarity(ingredients1, ingredients2):
    vectorizer = TfidfVectorizer().fit_transform([ingredients1, ingredients2])
    vectors = vectorizer.toarray()
    cosine_sim = cosine_similarity(vectors)
    return cosine_sim[0, 1]


# Function to evaluate generated recipe against a list of real recipes
def evaluate_generated_recipe_by_ingredients(generated_recipe, real_recipes, top_k=5):
    generated_ingredients = extract_ingredients(generated_recipe)

    similarities = []
    for real_recipe in tqdm(real_recipes, desc="Processing recipes"):
        real_ingredients = extract_ingredients(real_recipe)
        similarity = calculate_ingredient_similarity(generated_ingredients, real_ingredients)
        similarities.append((real_recipe, similarity))

    # Sort recipes based on ingredient similarity
    similarities.sort(key=lambda x: x[1], reverse=True)

    # Get top k recipes
    top_k_recipes = similarities[:top_k]

    results = []
    for real_recipe, sim in tqdm(top_k_recipes, desc="Calculating scores"):
        rouge_score = calculate_rouge_score(generated_recipe, real_recipe)
        gpt2_similarity = calculate_gpt2_similarity(generated_recipe, real_recipe, model, tokenizer)
        bert_similarity = calculate_bert_similarity(generated_recipe, real_recipe, model_bert, tokenizer_bert)
        results.append((real_recipe, sim, rouge_score, gpt2_similarity, bert_similarity))

    return results

In [13]:
# Evaluate and print top k recipes
top_k = 5
top_k_recipes = evaluate_generated_recipe_by_ingredients(generated_recipe, recipe_list, top_k=top_k)

for i, (recipe, ingredient_sim, rouge_score, gpt2_sim, bert_sim) in enumerate(top_k_recipes):
    print(f"\nRecipe {i + 1} (Ingredient Similarity: {ingredient_sim:.2f}):")
    print(print_highlighted(recipe, ingredients))
    print(f"ROUGE-L F1: {rouge_score:.4f}")
    print(f"GPT-2 Similarity: {gpt2_sim:.4f}")
    print(f"BERT Similarity: {bert_sim:.4f}")

Calculating scores: 100%|██████████| 5/5 [00:01<00:00,  3.35it/s]


Recipe 1 (Ingredient Similarity: 0.26):
[BOS]ground beef, fresh mushrooms, celery, tomato paste, tomato sauce, rigatoni pasta, cheddar cheese, cream of mushroom soup[STEPS]brown ground beef , drain well, add tomato paste , tomato sauce and favorite seasonings, layer half of rigatoni noodles on bottom of baking dish, top with half of meat sauce, repeat layers, top with cheese and mushroom soup, bake 1 1 / 2 hours at 300[EOS]
ROUGE-L F1: 0.1124
GPT-2 Similarity: 0.9995
BERT Similarity: 0.9414

Recipe 2 (Ingredient Similarity: 0.22):
[BOS]cream, tasty cheese, onion, garlic clove, nam pla, tomato paste, fish fillet[STEPS]make sauce by combining all ingredients in saucepan over medium heat until cheese is melted, place fish fillets into baking dish, spoon the sauce onto the fillets, bake at 425 - 450 degrees for 10-15 minutes , or under the griller until cooked and top is browned, fish should flake easily with a fork, "on the other hand , it shouldnt look dried out either", alternatively ,